# Kazi ya Uainishaji wa Maandishi

Katika moduli hii, tutaanza na kazi rahisi ya uainishaji wa maandishi kwa kutumia seti ya data ya **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: tutatambua vichwa vya habari vya habari katika mojawapo ya makundi 4: Dunia, Michezo, Biashara, na Sayansi/Tecknolojia.

## Seti ya Data

Ili kupakia seti ya data, tutatumia API ya **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Sasa tunaweza kufikia sehemu za mafunzo na majaribio za seti ya data kwa kutumia `dataset['train']` na `dataset['test']` mtawaliwa:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Hebu tuchapishe vichwa vya habari 10 vya kwanza kutoka kwenye seti yetu ya data:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Uwakilishi wa maandishi kwa namba

Sasa tunahitaji kubadilisha maandishi kuwa **namba** ambazo zinaweza kuwakilishwa kama tensors. Ikiwa tunataka uwakilishi wa kiwango cha maneno, tunahitaji kufanya mambo mawili:

* Tumia **tokenizer** kugawanya maandishi kuwa **tokeni**.
* Unda **msamiati** wa tokeni hizo.

### Kuweka kikomo kwa ukubwa wa msamiati

Katika mfano wa dataset ya AG News, ukubwa wa msamiati ni mkubwa sana, zaidi ya maneno 100k. Kwa ujumla, hatuhitaji maneno ambayo yanapatikana mara chache sana katika maandishi — sentensi chache tu zitakuwa nayo, na modeli haitajifunza kutoka kwao. Kwa hivyo, ina mantiki kuweka kikomo kwa ukubwa wa msamiati kwa idadi ndogo kwa kupitisha hoja kwa mjenzi wa vectorizer:

Hatua zote mbili zinaweza kushughulikiwa kwa kutumia safu ya **TextVectorization**. Hebu tuunde kitu cha vectorizer, kisha tuitie njia ya `adapt` ili kupitia maandishi yote na kujenga msamiati:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Kumbuka** tunatumia sehemu tu ya dataset nzima kujenga msamiati. Tunafanya hivi ili kuharakisha muda wa utekelezaji na kuepuka kukufanya usubiri. Hata hivyo, tunachukua hatari kwamba baadhi ya maneno kutoka dataset nzima hayatajumuishwa kwenye msamiati, na yatapuuzwa wakati wa mafunzo. Kwa hivyo, kutumia ukubwa wa msamiati mzima na kupitia dataset yote wakati wa `adapt` kunapaswa kuongeza usahihi wa mwisho, lakini si kwa kiwango kikubwa.

Sasa tunaweza kufikia msamiati halisi:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Kutumia vectorizer, tunaweza kwa urahisi kuweka maandishi yoyote katika seti ya nambari:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Uwakilishi wa maandishi kwa kutumia Bag-of-words

Kwa sababu maneno yanawakilisha maana, wakati mwingine tunaweza kuelewa maana ya kipande cha maandishi kwa kuangalia tu maneno binafsi, bila kujali mpangilio wao katika sentensi. Kwa mfano, wakati wa kuainisha habari, maneno kama *hali ya hewa* na *theluji* yanaweza kuashiria *utabiri wa hali ya hewa*, wakati maneno kama *hisa* na *dola* yangeelekea kuelekea *habari za kifedha*.

**Bag-of-words** (BoW) ni uwakilishi wa vekta wa jadi ambao ni rahisi zaidi kueleweka. Kila neno linaunganishwa na faharasa ya vekta, na kipengele cha vekta kinaonyesha idadi ya mara neno fulani linavyotokea katika hati fulani.

![Picha inayoonyesha jinsi uwakilishi wa vekta wa bag-of-words unavyohifadhiwa kwenye kumbukumbu.](../../../../../translated_images/sw/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Unaweza pia kufikiria BoW kama jumla ya vekta zote za one-hot-encoded kwa maneno binafsi katika maandishi.

Hapo chini kuna mfano wa jinsi ya kuunda uwakilishi wa bag-of-words kwa kutumia maktaba ya python ya Scikit Learn:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Tunaweza pia kutumia Keras vectorizer tuliyofafanua hapo juu, kubadilisha kila nambari ya neno kuwa one-hot encoding na kuongeza vektori zote hizo:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Kumbuka**: Unaweza kushangaa kwamba matokeo yanatofautiana na mfano wa awali. Sababu ni kwamba katika mfano wa Keras urefu wa vector unalingana na ukubwa wa msamiati, ambao ulijengwa kutoka kwenye seti nzima ya data ya AG News, ilhali katika mfano wa Scikit Learn tulijenga msamiati kutoka kwenye maandishi ya sampuli papo hapo.


## Kufundisha Kainishaji wa BoW

Sasa kwa kuwa tumejifunza jinsi ya kujenga uwakilishi wa mfuko-wa-maneno kwa maandishi yetu, hebu tufundishe kainishaji inayoutumia. Kwanza, tunahitaji kubadilisha seti yetu ya data kuwa uwakilishi wa mfuko-wa-maneno. Hii inaweza kufanyika kwa kutumia kazi ya `map` kwa njia ifuatayo:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Sasa hebu tueleze mtandao wa neva wa kuainisha rahisi ambao una safu moja ya mstari. Ukubwa wa ingizo ni `vocab_size`, na ukubwa wa matokeo unahusiana na idadi ya madarasa (4). Kwa sababu tunatatua kazi ya uainishaji, kazi ya mwisho ya uanzishaji ni **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Kwa kuwa tuna madarasa 4, usahihi wa zaidi ya 80% ni matokeo mazuri.

## Kufundisha classifier kama mtandao mmoja

Kwa sababu vectorizer pia ni safu ya Keras, tunaweza kufafanua mtandao unaojumuisha vectorizer, na kuufundisha kutoka mwanzo hadi mwisho. Kwa njia hii hatuhitaji kugeuza dataset kwa kutumia `map`, tunaweza tu kupitisha dataset ya asili kwenye ingizo la mtandao.

> **Note**: Bado tungehitaji kutumia `map` kwenye dataset yetu ili kubadilisha sehemu kutoka kwa kamusi (kama `title`, `description` na `label`) kuwa tuples. Hata hivyo, tunapopakia data kutoka diski, tunaweza kujenga dataset yenye muundo unaohitajika moja kwa moja.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrams, trigrams na n-grams

Kikwazo kimoja cha mbinu ya bag-of-words ni kwamba baadhi ya maneno ni sehemu ya misemo ya maneno mengi, kwa mfano, neno 'hot dog' lina maana tofauti kabisa na maneno 'hot' na 'dog' katika muktadha mwingine. Ikiwa tutawakilisha maneno 'hot' na 'dog' kila mara kwa kutumia vekta zile zile, inaweza kuichanganya modeli yetu.

Ili kushughulikia hili, **uwakilishi wa n-gram** mara nyingi hutumika katika mbinu za uainishaji wa nyaraka, ambapo marudio ya kila neno, maneno mawili au maneno matatu ni kipengele muhimu kwa kufundisha viainishi. Katika uwakilishi wa bigram, kwa mfano, tutaongeza jozi zote za maneno kwenye msamiati, pamoja na maneno ya asili.

Hapa chini kuna mfano wa jinsi ya kuunda uwakilishi wa bigram bag-of-words kwa kutumia Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Hasara kuu ya mbinu ya n-gram ni kwamba ukubwa wa msamiati huanza kukua kwa kasi sana. Kwa vitendo, tunahitaji kuchanganya uwakilishi wa n-gram na mbinu ya kupunguza vipimo, kama vile *embeddings*, ambayo tutajadili katika kipengele kinachofuata.

Ili kutumia uwakilishi wa n-gram katika seti yetu ya data ya **AG News**, tunahitaji kupitisha kipengele cha `ngrams` kwa mjenzi wetu wa `TextVectorization`. Urefu wa msamiati wa bigram ni **kubwa sana**, katika hali yetu ni zaidi ya tokeni milioni 1.3! Kwa hivyo, inafaa kupunguza tokeni za bigram kwa idadi fulani ya busara.

Tunaweza kutumia msimbo ule ule kama hapo juu kufundisha classifier, hata hivyo, hii haitakuwa na ufanisi wa kumbukumbu. Katika kipengele kinachofuata, tutafundisha classifier ya bigram kwa kutumia embeddings. Kwa sasa, unaweza kujaribu mafunzo ya classifier ya bigram katika daftari hili na uone kama unaweza kupata usahihi wa juu zaidi.


## Kuhesabu Vekta za BoW Kiotomatiki

Katika mfano hapo juu tulihesabu vekta za BoW kwa mkono kwa kujumlisha usimbaji wa moja kwa moja wa maneno ya kibinafsi. Hata hivyo, toleo la hivi karibuni la TensorFlow linaturuhusu kuhesabu vekta za BoW kiotomatiki kwa kupitisha kipengele `output_mode='count` kwenye mjenzi wa vectorizer. Hii inafanya kufafanua na kufundisha modeli yetu kuwa rahisi zaidi:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Mzunguko wa Neno - Mzunguko wa Nyaraka Kinyume (TF-IDF)

Katika uwakilishi wa BoW, matukio ya maneno yanapimwa kwa kutumia mbinu ile ile bila kujali neno lenyewe. Hata hivyo, ni wazi kwamba maneno yanayojirudia mara kwa mara kama *a* na *in* yana umuhimu mdogo sana kwa uainishaji ikilinganishwa na maneno maalum. Katika kazi nyingi za NLP, baadhi ya maneno ni muhimu zaidi kuliko mengine.

**TF-IDF** inasimama kwa **mzunguko wa neno - mzunguko wa nyaraka kinyume**. Ni toleo la bag-of-words, ambapo badala ya thamani ya 0/1 ya binary inayoonyesha uwepo wa neno katika nyaraka, thamani ya namba ya desimali hutumika, ambayo inahusiana na mzunguko wa neno hilo katika mkusanyiko.

Kwa ufafanuzi rasmi zaidi, uzito $w_{ij}$ wa neno $i$ katika nyaraka $j$ hufafanuliwa kama:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
ambapo
* $tf_{ij}$ ni idadi ya matukio ya $i$ katika $j$, yaani thamani ya BoW tuliyoiona awali
* $N$ ni idadi ya nyaraka katika mkusanyiko
* $df_i$ ni idadi ya nyaraka zinazojumuisha neno $i$ katika mkusanyiko mzima

Thamani ya TF-IDF $w_{ij}$ huongezeka kulingana na idadi ya mara neno linavyoonekana katika nyaraka na hupunguzwa na idadi ya nyaraka katika mkusanyiko ambazo zinajumuisha neno hilo, jambo ambalo husaidia kurekebisha ukweli kwamba baadhi ya maneno huonekana mara nyingi zaidi kuliko mengine. Kwa mfano, ikiwa neno linaonekana katika *kila* nyaraka katika mkusanyiko, $df_i=N$, na $w_{ij}=0$, na maneno hayo yatapuuzwa kabisa.

Unaweza kuunda kwa urahisi uakisi wa maandishi wa TF-IDF kwa kutumia Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

Katika Keras, safu ya `TextVectorization` inaweza kuhesabu mara kwa mara za TF-IDF kiotomatiki kwa kupitisha kipengele `output_mode='tf-idf'`. Hebu turudie msimbo tuliotumia hapo juu ili kuona kama kutumia TF-IDF kunaongeza usahihi:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Hitimisho

Ingawa uwakilishi wa TF-IDF unatoa uzito wa marudio kwa maneno tofauti, hauwezi kuwakilisha maana au mpangilio. Kama mwanaisimu maarufu J. R. Firth alivyosema mwaka 1935, "Maana kamili ya neno daima ni ya muktadha, na hakuna uchambuzi wa maana bila muktadha unaoweza kuchukuliwa kwa uzito." Tutajifunza jinsi ya kunasa taarifa za muktadha kutoka kwa maandishi kwa kutumia uundaji wa lugha baadaye katika kozi.



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya kutafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
